In [2]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached torch-2.3.1-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.1-cp312-cp312-win_amd64.whl (159.7 MB)
Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
Using cached intel_openmp-2021.4.0-py2.py3-non

In [9]:
import logging
import torch
import json
from transformers import BertTokenizer, BertModel, DPRQuestionEncoderTokenizer, DPRQuestionEncoder

In [13]:
from transformers import BertTokenizer, BertModel, DPRQuestionEncoderTokenizer, DPRQuestionEncoder

In [14]:
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
def initialize_bert():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    return tokenizer, model

def initialize_dpr():
    dpr_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
    dpr_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
    return dpr_tokenizer, dpr_model


In [15]:

def generate_bert_embeddings(tree_root, tokenizer, model):
    node_to_embedding = {}

    def traverse_tree(node):
        # Log the node being processed
        logging.info(f"Processing node: {node}")
        
        try:
            if hasattr(node, 'content') and node.content:
                inputs = tokenizer(node.content, return_tensors='pt', truncation=True, padding=True, max_length=512)
                with torch.no_grad():
                    outputs = model(**inputs)
                if hasattr(node, 'id'):
                    logging.info(f"Node ID: {node.id}, Content: {node.content[:30]}...")  # Log a portion of the content for readability
                    node_to_embedding[node.id] = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
                else:
                    logging.warning(f"Node {node} does not have an 'id' attribute.")
            else:
                logging.warning(f"Node {node} does not have 'content' or content is empty.")

            for child in node.children:
                traverse_tree(child)
        except Exception as e:
            logging.error(f"Error processing node {node}: {e}")

    traverse_tree(tree_root)
    return node_to_embedding

def generate_dpr_embeddings(tree_root, dpr_tokenizer, dpr_model):
    node_to_embedding = {}

    def traverse_tree(node):
        # Log the node being processed
        logging.info(f"Processing node: {node}")

        try:
            if hasattr(node, 'content') and node.content:
                inputs = dpr_tokenizer(node.content, return_tensors='pt', truncation=True, padding=True, max_length=512)
                with torch.no_grad():
                    outputs = dpr_model(**inputs)
                if hasattr(node, 'id'):
                    logging.info(f"Node ID: {node.id}, Content: {node.content[:30]}...")  # Log a portion of the content for readability
                    node_to_embedding[node.id] = outputs.pooler_output.squeeze().detach().numpy()
                else:
                    logging.warning(f"Node {node} does not have an 'id' attribute.")
            else:
                logging.warning(f"Node {node} does not have 'content' or content is empty.")

            for child in node.children:
                traverse_tree(child)
        except Exception as e:
            logging.error(f"Error processing node {node}: {e}")

    traverse_tree(tree_root)
    return node_to_embedding

# Example of setting up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Assuming tree_root, tokenizer, model, dpr_tokenizer, and dpr_model are defined elsewhere
# bert_embeddings = generate_bert_embeddings(tree_root, tokenizer, model)
# dpr_embeddings = generate_dpr_embeddings(tree_root, dpr_tokenizer, dpr_model)
file_path=r"hierarchical_tree.json"

with open(file_path, 'r') as file:
    data = json.load(file)

# Initialize BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Initialize DPR model and tokenizer
dpr_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
dpr_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

# Generate embeddings
bert_embeddings = generate_bert_embeddings(data, bert_tokenizer, bert_model)
dpr_embeddings = generate_dpr_embeddings(data, dpr_tokenizer, dpr_model)

# Verify if embeddings are generated



e:\aaaaa\bbot1\myenv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--facebook--dpr-question_encoder-single-nq-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used whe

In [2]:
import json
file_path=r"E:\aaaaa\bbot1\hierarchical_tree.json"
with open(file_path, 'r',encoding='utf-8') as file:
    data = json.load(file)

In [2]:
id=data['id']
print(f"id:{id}")

id:0


In [1]:
import json
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def replace_empty_content(tree_root, replacement_value="alpha"):
    def traverse_and_replace(node):
        logging.info(f"Processing node with ID: {node.get('id', 'No ID')}")

        try:
            # Check if 'content' is empty or None
            if not node.get('content'):
                logging.info(f"Node with ID {node.get('id', 'No ID')} has empty 'content'. Replacing with '{replacement_value}'.")
                node['content'] = "No content"
            
            # Traverse children if any
            children = node.get('children', [])
            for child in children:
                traverse_and_replace(child)
        except Exception as e:
            logging.error(f"Error processing node with ID {node.get('id', 'No ID')}: {e}")

    traverse_and_replace(tree_root)

# Load JSON data
file_path = r'E:\aaaaa\bbot1\hierarchical_tree.json'  # Update with your actual file path

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Replace empty content
replace_empty_content(data)

# Save the updated JSON data
output_file_path = 'updated_hierarchical_tree.json'  # Update with your desired output file path

with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

logging.info(f"Updated JSON data saved to {output_file_path}")


2024-07-26 19:33:28,283 - INFO - Processing node with ID: 0
2024-07-26 19:33:28,283 - INFO - Updated JSON data saved to updated_hierarchical_tree.json


In [11]:
def read_toc_from_file(file_path):
    toc = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                
                if not line:  # Skip empty lines
                    continue
                
                # Split by comma and handle lines that might have extra commas or quotes
                parts = line.split(',')
                
                if len(parts) < 3:  # Skip lines with fewer than 3 parts
                    print(f"Skipping invalid line (too few parts): {line}")
                    continue
                
                # Extract level, title, and page number
                level_str = parts[0].strip()
                title = ','.join(parts[1:-1]).strip()  # Join any additional commas in the title
                page_num_str = parts[-1].strip()
                
                try:
                    level = int(level_str)
                    page_num = int(page_num_str)
                    toc.append((level, title, page_num))
                except ValueError as e:
                    print(f"Error processing line '{line}': {e}")
                    continue
    except FileNotFoundError as e:
        print(f"Error: {e}")
    return toc

In [12]:
toc_path = "uploads/toc.txt"

toc = read_toc_from_file(toc_path)

Skipping invalid line (too few parts): 2,
Skipping invalid line (too few parts): Chapter 14. Deep Computer Vision Using Convolutional Neural Networks,
Skipping invalid line (too few parts): 457


In [13]:
toc

[(1, 'Cover', 1),
 (1, 'Copyright', 4),
 (1, 'Table of Contents', 5),
 (1, 'Preface', 13),
 (2, 'The Machine Learning Tsunami', 13),
 (2, 'Machine Learning in Your Projects', 13),
 (2, 'Objective and Approach', 14),
 (2, 'Prerequisites', 15),
 (2, 'Roadmap', 15),
 (2, 'Other Resources', 17),
 (2, 'Conventions Used in This Book', 18),
 (2, 'Code Examples', 19),
 (2, 'Using Code Examples', 20),
 (2, 'O’Reilly Safari', 20),
 (2, 'How to Contact Us', 20),
 (2, 'Changes in the Second Edition', 21),
 (2, 'Acknowledgments', 25),
 (1, 'Part I. The Fundamentals of Machine Learning', 27),
 (2, 'Chapter 1. The Machine Learning Landscape', 29),
 (3, 'What Is Machine Learning?', 30),
 (3, 'Why Use Machine Learning?', 30),
 (3, 'Types of Machine Learning Systems', 34),
 (4, 'Supervised/Unsupervised Learning', 34),
 (4, 'Batch and Online Learning', 41),
 (4, 'Instance-Based Versus Model-Based Learning', 44),
 (3, 'Main Challenges of Machine Learning', 50),
 (4, 'Insufficient Quantity of Training Data